# Originally adapted from IT_DEMO_MODEL_DEV

In [1]:
%run ./model_end_events_for_outages_METHODS.ipynb

In [2]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

#---------------------------------------------------------------------
import sys, os
import re
from pathlib import Path
import json
import pickle
import joblib

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import random

import copy
import itertools
import adjustText

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
from MeterPremise import MeterPremise
from EEMSP import EEMSP
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIEDE_DEV import AMIEDE_DEV
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
from OutagePredictor import OutagePredictor
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer
#---------------------------------------------------------------------

In [3]:
model_base_dir = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20231201\Models\All_EEMSP_agg_Top10_v4'

In [4]:
prediction_date   = pd.to_datetime('2023-07-27')

In [5]:
tmp_f = open(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20231201\Models\All_EEMSP_agg_Top10_v4\Slices\ledger.json')
ledger = json.load(tmp_f)
tmp_f.close()
trsf_pole_nbs_model = natsorted(ledger.keys())

In [6]:
random.seed(42)

In [7]:
self = OutagePredictor(prediction_date=prediction_date)
#-----
self.set_model_dir(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20231201\Models\All_EEMSP_agg_Top10_v4')
#-----
self.set_trsf_pole_nbs(trsf_pole_nbs = random.sample(trsf_pole_nbs_model, 1000))
print(f"n_trsf_pole_nbs = {len(self.trsf_pole_nbs)}")
print(f"\nSQL statement:\n{self.trsf_pole_nbs_sql}")
#-----
self.initialize_data(
    evsSum_sql_fcn=AMIEndEvents_SQL.build_sql_end_events,  
    evsSum_sql_kwargs=dict(opco='oh'), 
)

n_trsf_pole_nbs = 1000

SQL statement:



OperationalError: ('08S01', '[08S01] [Simba][DriverIAMSupport] (8600) Connection Error: Authentication failed on the Azure server. Please check the IdP Tenant, User, Password, Client Secret and Client ID. Response code: 400 (8600) (SQLDriverConnect)')

In [ ]:
# Make predictions
y_pred = self.model_clf.predict(self.X_test)

In [ ]:
y_scores = self.model_clf.predict_proba(self.X_test)

In [8]:
# Sanity check
threshold=0.5
y_pred_from_proba = [int(x>threshold) for x in y_scores[:, 1]]
assert(all(y_pred_from_proba==y_pred))

NameError: name 'y_scores' is not defined

In [ ]:
[x for x in y_scores[:, 1]]==[x[1] for x in y_scores]

In [ ]:
y_pred.sum()

In [ ]:
trsf_pole_nbs = self.trsf_pole_nbs

In [ ]:
# trsf_pole_nbs

In [ ]:
def build_ledger_inv(
    ledger, 
    return_type = pd.Series, 
    trsf_pole_nbs = None
):
    r"""
    Input:
        keys   = trsf_pole_nbs
        values = model identifiers

    Output:
        return_type == pd.Series
            a pd.Series object with index equal to model identifier and values equal to lists of trsf_pole_nbs
        return_type == dict
            keys   = model identifiers
            values = lists of trsf_pole_nbs covered by model

    trsf_pole_nbs:
        If None, simply inver ledger
        If supplied, ensure they are contained in ledger, and then return inverted version of ledger only containing the 
          poles contained in trsf_pole_nbs
    """
    #-------------------------
    assert(return_type in [pd.Series, dict])
    #-------------------------
    ledger_srs = pd.Series(ledger)
    #-------------------------
    if trsf_pole_nbs is not None:
        assert(set(trsf_pole_nbs).difference(set(ledger.keys()))==set())
        ledger_srs = ledger_srs.loc[trsf_pole_nbs]
    #-------------------------
    # Invert the series, so indices are model ids and values are trsf_pole_nbs
    # AND group by model id and collect trsf_pole_nbs in lists
    ledger_inv = pd.Series(ledger_srs.index.values, index=ledger_srs).groupby(level=0).apply(list)
    #-------------------------
    if return_type==dict:
        ledger_inv = ledger_inv.to_dict()
    #-------------------------
    return ledger_inv


def group_trsf_pole_nbs_by_model(
    trsf_pole_nbs, 
    ledger
):
    r"""
    Returns a pd.Series object with index equal to model identifier and values equal to lists of trsf_pole_nbs
    """
    #-------------------------
    ledger_inv = build_ledger_inv(
        ledger        = ledger, 
        return_type   = pd.Series, 
        trsf_pole_nbs = trsf_pole_nbs        
    )
    #-------------------------
    return ledger_inv

In [ ]:
ledger_inv_srs = group_trsf_pole_nbs_by_model(
    trsf_pole_nbs = trsf_pole_nbs, 
    ledger        = ledger
)

In [ ]:
ledger_inv_srs

In [ ]:
model_i = 7
trsf_poles_i = ledger_inv_srs[model_i]

print(model_i)
model_i_dir = os.path.join(model_base_dir, 'Slices', str(model_i))
assert(os.path.isdir(model_i_dir))
#-------------------------
# If no model able to be made for group, directory will be empty
# if not os.listdir(model_i_dir):
#     print('No model compiled.\nContinuing to next')
#     continue
#-------------------------
predictor_i = OutagePredictor(prediction_date=prediction_date)
predictor_i.set_model_dir(model_i_dir)
predictor_i.set_trsf_pole_nbs(trsf_poles_i)
predictor_i.initialize_data(
    evsSum_sql_fcn=AMIEndEvents_SQL.build_sql_end_events,  
    evsSum_sql_kwargs=dict(opco='oh'), 
)
#-------------------------
# assert(model_i not in predictors.keys())
# predictors[model_i] = predictor_i

In [ ]:
predictor_i.evsSum_df.shape[0]

In [ ]:
len(list(set(predictor_i.trsf_pole_nbs).difference(set(predictor_i.evsSum_df['trsf_pole_nb'].unique()))))

In [ ]:
list(set(predictor_i.trsf_pole_nbs).difference(set(predictor_i.evsSum_df['trsf_pole_nb'].unique())))

In [ ]:
self = predictor_i

# date          = '2023-07-26'
# date = (self.prediction_date - 0.5*(self.idk_name_1+self.idk_name_2)).date()
date = self.prediction_date - self.idk_name_2
evsSum_sql_kwargs = dict(opco='oh')
date_col='aep_event_dt'

In [ ]:
def build_zero_counts_evsSum_df_OLD(
    self, 
    date_col          = 'aep_event_dt', 
    verbose           = True
):
    r"""
    Any transformer whose meters did not register events will not be retrieved by the query.
        Therefore, they must be added by hand.
        That is the purpose of this function.

    To avoid any more confusion than necessary, self.__evsSum_df will contain only meters registering events
        This is natural, as the SQL query will only return such elements.
        Therefore, the zero_counts_df built here will not be appended to self.__evsSum_df.

    This function will produce a pd.DataFrame object of the same form as self.__evsSum_df for the transformers whose meters
      did not register an event.
    Only one single date is needed for each transformer, the function OutagePredictor.build_rcpx_from_evsSum_df will handle the rest, as usual.
    IMPORTANT NOTE: It doesn't really matter what this single date is, AS LONG AS IT FALLS WITHIN THE WINDOW OF INTEREST.
                      i.e., the date must be between start=prediction_date-td_max and end=prediction_date-td_min.
                    However, it does slightly matter, as the function OutagePredictor.get_xfmr_meter_cnt_closest_to_date will be used to find the meter counts for
                      the transformers, and therefore using the date closest to the prediction date is likely to best choice (although, the meter count
                      is likely stable during the window of interest, which is why it doesn't really matter much)
    """
    #--------------------------------------------------
    zero_counts_xfmrs = list(set(self.trsf_pole_nbs).difference(set(self.evsSum_df['trsf_pole_nb'].unique())))
    #-------------------------
    if len(zero_counts_xfmrs)==0:
        # return pd.DataFrame()
        return None
    #-------------------------
    date              = self.prediction_date - self.idk_name_2
    evsSum_sql_kwargs = self.evsSum_sql_kwargs
    #-----
    # Let trsf_pole_nbs, cols_of_interest, and date (and, therefore, date_range) be handled by OutagePredictor.get_xfmr_meter_cnt_closest_to_date
    keys_to_ignore = ['trsf_pole_nbs', 'trsf_pole_nb', 'cols_of_interest', 'date', 'date_range', 'datetime_range']
    evsSum_sql_kwargs = {k:v for k,v in evsSum_sql_kwargs.items() if k not in keys_to_ignore}
    #----------
    zero_counts_df = OutagePredictor.get_xfmr_meter_cnt_closest_to_date(
        trsf_pole_nbs      = zero_counts_xfmrs, 
        date               = date, 
        evsSum_sql_kwargs  = evsSum_sql_kwargs, 
        include_meter_cols = True, 
        batch_size         = 1000, 
        verbose            = verbose, 
        n_update           = 10, 
        conn_aws           = self.conn_aws, 
        return_sql         = False,
        keep_extra_cols    = False
    )
    #-------------------------
    tmp_idx_col = Utilities.generate_random_string()
    zero_counts_df[tmp_idx_col] = list(range(self.evsSum_df.shape[0], self.evsSum_df.shape[0]+zero_counts_df.shape[0]))
    zero_counts_df = zero_counts_df.set_index(tmp_idx_col, drop=True)
    zero_counts_df.index.name = None
    #--------------------------------------------------
    addtln_zero_counts_df_cols = pd.DataFrame(
        data    = 0, 
        index   = list(range(self.evsSum_df.shape[0], self.evsSum_df.shape[0]+zero_counts_df.shape[0])), 
        columns = list(set(self.evsSum_df.columns).difference(set(zero_counts_df.columns)))
    )
    assert(all(zero_counts_df.index==addtln_zero_counts_df_cols.index))
    zero_counts_df = pd.concat([zero_counts_df, addtln_zero_counts_df_cols], axis=1, ignore_index=False)
    #-------------------------
    # As described above, only need each trsf_pole_nb to have a single entry within the window of interest, the fuction
    # OutagePredictor.project_time_pd_from_rcpx_0_and_prepare will handle zeroing everything else out
    zero_counts_df[date_col] = date
    #-----
    assert(set(self.evsSum_df.columns).symmetric_difference(set(zero_counts_df.columns))==set())
    zero_counts_df = zero_counts_df[self.evsSum_df.columns]
    #--------------------------------------------------
    return zero_counts_df

In [ ]:
# def build_zero_counts_evsSum_df(
#     trsf_pole_nbs, 
#     evsSum_df, 
#     date, 
#     evsSum_sql_kwargs = None, 
#     date_col          = 'aep_event_dt',
#     trsf_pole_nb_col  = 'trsf_pole_nb', 
#     conn_aws          = None, 
#     verbose           = True
# ):
#     r"""
#     Any transformer whose meters did not register events will not be retrieved by the query.
#         Therefore, they must be added by hand.
#         That is the purpose of this function.

#     Find any trsf_pole_nbs missing from evsSum_df, grab their meter counts info from events_summary_vw, and then populate
#       a pd.DataFrame of same structure as evsSum_df with all zeros for counts on date input  (intended to be self.prediction_date - self.idk_name_2).
#     NOTE: This originally was not a static function, hence the somewhat strange structure and documentation.
#     NOTE: keys_to_ignore (defined below) will be ignored if input in evsSum_sql_kwargs

#     To avoid any more confusion than necessary, self.__evsSum_df will contain only meters registering events
#         This is natural, as the SQL query will only return such elements.
#         Therefore, the zero_counts_df built here will not be appended to self.__evsSum_df.

#     This function will produce a pd.DataFrame object of the same form as self.__evsSum_df for the transformers whose meters
#       did not register an event.
#     Only one single date is needed for each transformer, the function OutagePredictor.build_rcpx_from_evsSum_df will handle the rest, as usual.
#     IMPORTANT NOTE: It doesn't really matter what this single date is, AS LONG AS IT FALLS WITHIN THE WINDOW OF INTEREST.
#                       i.e., the date must be between start=prediction_date-td_max and end=prediction_date-td_min.
#                     However, it does slightly matter, as the function OutagePredictor.get_xfmr_meter_cnt_closest_to_date will be used to find the meter counts for
#                       the transformers, and therefore using the date closest to the prediction date is likely to best choice (although, the meter count
#                       is likely stable during the window of interest, which is why it doesn't really matter much)
#     """
#     #--------------------------------------------------
#     zero_counts_xfmrs = list(set(trsf_pole_nbs).difference(set(evsSum_df[trsf_pole_nb_col].unique())))
#     #-------------------------
#     if len(zero_counts_xfmrs)==0:
#         # return pd.DataFrame()
#         return None
#     #-------------------------
#     if evsSum_sql_kwargs is None:
#         evsSum_sql_kwargs = dict()
#     assert(isinstance(evsSum_sql_kwargs, dict))
#     #-----
#     # Let trsf_pole_nbs, cols_of_interest, and date (and, therefore, date_range) be handled by OutagePredictor.get_xfmr_meter_cnt_closest_to_date
#     keys_to_ignore = ['trsf_pole_nbs', 'trsf_pole_nb', 'cols_of_interest', 'date', 'date_range', 'datetime_range']
#     evsSum_sql_kwargs = {k:v for k,v in evsSum_sql_kwargs.items() if k not in keys_to_ignore}
#     #----------
#     zero_counts_df = OutagePredictor.get_xfmr_meter_cnt_closest_to_date(
#         trsf_pole_nbs      = zero_counts_xfmrs, 
#         date               = date, 
#         evsSum_sql_kwargs  = evsSum_sql_kwargs, 
#         include_meter_cols = True, 
#         batch_size         = 1000, 
#         verbose            = verbose, 
#         n_update           = 10, 
#         conn_aws           = conn_aws, 
#         return_sql         = False,
#         keep_extra_cols    = False
#     )
#     #-------------------------
#     tmp_idx_col = Utilities.generate_random_string()
#     zero_counts_df[tmp_idx_col] = list(range(evsSum_df.shape[0], evsSum_df.shape[0]+zero_counts_df.shape[0]))
#     zero_counts_df = zero_counts_df.set_index(tmp_idx_col, drop=True)
#     zero_counts_df.index.name = None
#     #--------------------------------------------------
#     addtln_zero_counts_df_cols = pd.DataFrame(
#         data    = 0, 
#         index   = list(range(evsSum_df.shape[0], evsSum_df.shape[0]+zero_counts_df.shape[0])), 
#         columns = list(set(evsSum_df.columns).difference(set(zero_counts_df.columns)))
#     )
#     assert(all(zero_counts_df.index==addtln_zero_counts_df_cols.index))
#     zero_counts_df = pd.concat([zero_counts_df, addtln_zero_counts_df_cols], axis=1, ignore_index=False)
#     #-------------------------
#     # As described above, only need each trsf_pole_nb to have a single entry within the window of interest, the fuction
#     # OutagePredictor.project_time_pd_from_rcpx_0_and_prepare will handle zeroing everything else out
#     zero_counts_df[date_col] = date
#     #-----
#     assert(set(evsSum_df.columns).symmetric_difference(set(zero_counts_df.columns))==set())
#     zero_counts_df = zero_counts_df[evsSum_df.columns]
#     #--------------------------------------------------
#     return zero_counts_df

In [ ]:
#-------------------------
# 2b. Any transformer whose meters did not register events will not be retrieved by the query
#       Therefore, they must be added by hand
#     To avoid any more confusion than necessary, self.__evsSum_df will contain only meters registering events
#       This is natural, as the SQL query will only return such elements.
#     However, if a 

zero_counts_xfmrs = list(set(self.trsf_pole_nbs).difference(set(self.evsSum_df['trsf_pole_nb'].unique())))
zero_counts_df = OutagePredictor.get_xfmr_meter_cnt_closest_to_date(
    trsf_pole_nbs      = zero_counts_xfmrs, 
    date               = date, 
    evsSum_sql_kwargs  = evsSum_sql_kwargs, 
    include_meter_cols = True, 
    batch_size         = 1000, 
    verbose            = True, 
    n_update           = 10, 
    conn_aws           = self.conn_aws, 
    return_sql         = False,
    keep_extra_cols    = False
)
#-------------------------
tmp_idx_col = Utilities.generate_random_string()
zero_counts_df[tmp_idx_col] = list(range(self.evsSum_df.shape[0], self.evsSum_df.shape[0]+zero_counts_df.shape[0]))
zero_counts_df = zero_counts_df.set_index(tmp_idx_col, drop=True)
zero_counts_df.index.name = None
#--------------------------------------------------
addtln_zero_counts_df_cols = pd.DataFrame(
    data    = 0, 
    index   = list(range(self.evsSum_df.shape[0], self.evsSum_df.shape[0]+zero_counts_df.shape[0])), 
    columns = list(set(self.evsSum_df.columns).difference(set(zero_counts_df.columns)))
)
assert(all(zero_counts_df.index==addtln_zero_counts_df_cols.index))
zero_counts_df = pd.concat([zero_counts_df, addtln_zero_counts_df_cols], axis=1, ignore_index=False)
#-------------------------
# As described above, only need each trsf_pole_nb to have a single entry within the window of interest, the fuction
# OutagePredictor.project_time_pd_from_rcpx_0_and_prepare will handle zeroing everything else out
zero_counts_df[date_col] = date
#-----
assert(set(self.evsSum_df.columns).symmetric_difference(set(zero_counts_df.columns))==set())
zero_counts_df = zero_counts_df[self.evsSum_df.columns]

In [ ]:
zero_counts_df_NEW_OLD = build_zero_counts_evsSum_df_OLD(
    self              = self, 
    date_col          = 'aep_event_dt', 
    verbose           = True
)

In [ ]:
zero_counts_df_NEW = OutagePredictor.build_zero_counts_evsSum_df(
    trsf_pole_nbs     = self.trsf_pole_nbs, 
    evsSum_df         = self.evsSum_df, 
    date              = self.prediction_date - self.idk_name_2, 
    evsSum_sql_kwargs = self.evsSum_sql_kwargs, 
    date_col          = 'aep_event_dt',
    trsf_pole_nb_col  = 'trsf_pole_nb', 
    conn_aws          = self.conn_aws, 
    verbose           = True
)

In [ ]:
zero_counts_df

In [ ]:
zero_counts_df_NEW_OLD

In [ ]:
zero_counts_df_NEW

In [ ]:
zero_counts_df.equals(zero_counts_df_NEW)

In [ ]:
zero_counts_df.sort_values(by='trsf_pole_nb', ignore_index=True).drop(columns=['serialnumber', 'aep_premise_nb']).equals(zero_counts_df_NEW.sort_values(by='trsf_pole_nb', ignore_index=True).drop(columns=['serialnumber', 'aep_premise_nb']))

In [ ]:
prediction_dates = pd.date_range(
    start = pd.to_datetime('2023-05-02'), 
    end   = pd.to_datetime('2023-09-30'), 
    freq  = pd.Timedelta('1D')
)
prediction_dates

In [ ]:
date_ranges = OutagePredictor.get_date_ranges_for_predictions(
    prediction_dates  = prediction_dates, 
    predictions_range = None, 
    idk_name_1 = self.idk_name_1, 
    idk_name_2 = self.idk_name_2
)

In [ ]:
date_ranges

In [ ]:
v0 = OutagePredictor.build_rcpx_from_evsSum_df(
            evsSum_df                   = self.evsSum_df, 
            data_structure_df           = self.data_structure_df, 
            prediction_date             = self.prediction_date, 
            td_min                      = self.idk_name_2, 
            td_max                      = self.idk_name_1, 
            cr_trans_dict               = self.cr_trans_dict, 
            freq                        = '5D', 
            group_cols                  = ['trsf_pole_nb'], 
            date_col                    = 'aep_event_dt', 
            normalize_by_SNs            = True, 
            normalize_by_time           = True, 
            include_power_down_minus_up = False, 
            regex_patterns_to_remove    = ['.*cleared.*', '.*Test Mode.*'], 
            combine_cpo_df_reasons      = True, 
            xf_meter_cnt_col            = 'xf_meter_cnt', 
            events_tot_col              = 'events_tot', 
            trsf_pole_nb_col            = 'trsf_pole_nb', 
            other_reasons_col           = 'Other Reasons', 
            total_counts_col            = 'total_counts', 
            nSNs_col                    = 'nSNs'
        )

In [ ]:
v0

In [ ]:
supplmnt = OutagePredictor.build_rcpx_from_evsSum_df(
            evsSum_df                   = zero_counts_df, 
            data_structure_df           = self.data_structure_df, 
            prediction_date             = self.prediction_date, 
            td_min                      = self.idk_name_2, 
            td_max                      = self.idk_name_1, 
            cr_trans_dict               = self.cr_trans_dict, 
            freq                        = '5D', 
            group_cols                  = ['trsf_pole_nb'], 
            date_col                    = 'aep_event_dt', 
            normalize_by_SNs            = True, 
            normalize_by_time           = True, 
            include_power_down_minus_up = False, 
            regex_patterns_to_remove    = ['.*cleared.*', '.*Test Mode.*'], 
            combine_cpo_df_reasons      = True, 
            xf_meter_cnt_col            = 'xf_meter_cnt', 
            events_tot_col              = 'events_tot', 
            trsf_pole_nb_col            = 'trsf_pole_nb', 
            other_reasons_col           = 'Other Reasons', 
            total_counts_col            = 'total_counts', 
            nSNs_col                    = 'nSNs'
        )

In [ ]:
supplmnt

In [ ]:
v0.columns.tolist()==supplmnt.columns.tolist()

In [ ]:
if self.evsSum_df.shape[0]>0:
    assert(all(zero_counts_df.columns==self.evsSum_df.columns))
self.evsSum_df = pd.concat([self.evsSum_df, zero_counts_df], axis=0, ignore_index=False)

In [ ]:
assert(all(zero_counts_df.columns==self.evsSum_df.columns))

In [ ]:
# self.evsSum_df = pd.concat([self.evsSum_df, zero_counts_df], axis=0, ignore_index=False)
evsSum_df = pd.concat([self.evsSum_df, zero_counts_df], axis=0, ignore_index=False)

In [ ]:
v1 = OutagePredictor.build_rcpx_from_evsSum_df(
            evsSum_df                   = evsSum_df, 
            data_structure_df           = self.data_structure_df, 
            prediction_date             = self.prediction_date, 
            td_min                      = self.idk_name_2, 
            td_max                      = self.idk_name_1, 
            cr_trans_dict               = self.cr_trans_dict, 
            freq                        = '5D', 
            group_cols                  = ['trsf_pole_nb'], 
            date_col                    = 'aep_event_dt', 
            normalize_by_SNs            = True, 
            normalize_by_time           = True, 
            include_power_down_minus_up = False, 
            regex_patterns_to_remove    = ['.*cleared.*', '.*Test Mode.*'], 
            combine_cpo_df_reasons      = True, 
            xf_meter_cnt_col            = 'xf_meter_cnt', 
            events_tot_col              = 'events_tot', 
            trsf_pole_nb_col            = 'trsf_pole_nb', 
            other_reasons_col           = 'Other Reasons', 
            total_counts_col            = 'total_counts', 
            nSNs_col                    = 'nSNs'
        )

In [ ]:
v0

In [ ]:
v1

In [ ]:
prediction_dates = pd.date_range(
    start = pd.to_datetime('2023-09-01'), 
    end   = pd.to_datetime('2023-09-30'), 
    freq  = pd.Timedelta('1D')
)
prediction_dates

In [ ]:
outg_prdctr = OutagePredictor(prediction_date=prediction_dates[0])
#-----
outg_prdctr.set_model_dir(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20231201\Models\All_EEMSP_agg_Top10_v4')
outg_prdctr.set_trsf_pole_nbs(trsf_pole_nbs)

outg_prdctr.prep_multiple_prediction_dates(
    prediction_dates  = prediction_dates, 
    predictions_range = None, 
    evsSum_sql_fcn    = AMIEndEvents_SQL.build_sql_end_events,  
    evsSum_sql_kwargs = dict(opco='oh'), 
    save_args         = False
)

In [ ]:
outg_prdctr.evsSum_df['trsf_pole_nb'].nunique()

In [ ]:
len(trsf_pole_nbs)

In [ ]:
# Basically, each prediction range must include all trsf_pole_nbs

In [ ]:
date_col='aep_event_dt'
self = outg_prdctr

In [ ]:
def build_zero_counts_evsSum_df_multiple_pred_dates(
    trsf_pole_nbs, 
    evsSum_df, 
    prediction_dates, 
    td_min, 
    evsSum_sql_kwargs = None, 
    date_col          = 'aep_event_dt',
    trsf_pole_nb_col  = 'trsf_pole_nb', 
    conn_aws          = None, 
    verbose           = True    
):
    r"""
    """
    #--------------------------------------------------
    # Make sure date_col is datetime object
    evsSum_df[date_col] = pd.to_datetime(evsSum_df[date_col])
    
    zc_xfmrs_by_date = dict()
    for pred_date_i in prediction_dates:
        evsSum_df_i = evsSum_df[
            (evsSum_df[date_col] >= pred_date_i-self.idk_name_1) & 
            (evsSum_df[date_col] <= pred_date_i-self.idk_name_2)
        ]
        zero_counts_xfmrs_i = list(set(trsf_pole_nbs).difference(set(evsSum_df_i[trsf_pole_nb_col].unique())))
        if len(zero_counts_xfmrs_i)>0:
            assert(pred_date_i not in zc_xfmrs_by_date.keys())
            zc_xfmrs_by_date[pred_date_i] = zero_counts_xfmrs_i

In [ ]:
evsSum_df = self.evsSum_df

# Make sure date_col is datetime object
evsSum_df[date_col] = pd.to_datetime(evsSum_df[date_col])

zc_xfmrs_by_date = dict()
for pred_date_i in prediction_dates:
    evsSum_df_i = evsSum_df[
        (evsSum_df[date_col] >= pred_date_i-self.idk_name_1) & 
        (evsSum_df[date_col] <= pred_date_i-self.idk_name_2)
    ]
    zero_counts_xfmrs_i = list(set(trsf_pole_nbs).difference(set(evsSum_df_i['trsf_pole_nb'].unique())))
    if len(zero_counts_xfmrs_i)>0:
        assert(pred_date_i not in zc_xfmrs_by_date.keys())
        zc_xfmrs_by_date[pred_date_i] = zero_counts_xfmrs_i

In [ ]:
len(zc_xfmrs_by_date)

In [ ]:
list(zc_xfmrs_by_date.keys())[0]

In [ ]:
pred_date_i = pd.Timestamp('2023-09-01 00:00:00')
zero_counts_xfmrs_i = zc_xfmrs_by_date[pred_date_i]
evsSum_df_i = evsSum_df[
    (evsSum_df[date_col] >= pred_date_i-self.idk_name_1) & 
    (evsSum_df[date_col] <= pred_date_i-self.idk_name_2)
]

In [ ]:
zero_counts_df_i = OutagePredictor.build_zero_counts_evsSum_df(
    trsf_pole_nbs     = zero_counts_xfmrs_i, 
    evsSum_df         = evsSum_df_i, 
    date              = pred_date_i - self.idk_name_2, 
    evsSum_sql_kwargs = self.evsSum_sql_kwargs, 
    date_col          = 'aep_event_dt',
    trsf_pole_nb_col  = 'trsf_pole_nb', 
    conn_aws          = self.conn_aws, 
    verbose           = True
)

In [ ]:
evsSum_df = self.evsSum_df

# Make sure date_col is datetime object
evsSum_df[date_col] = pd.to_datetime(evsSum_df[date_col])

zero_counts_df = pd.DataFrame()
for pred_date_i in prediction_dates:
    evsSum_df_i = evsSum_df[
        (evsSum_df[date_col] >= pred_date_i-self.idk_name_1) & 
        (evsSum_df[date_col] <= pred_date_i-self.idk_name_2)
    ]
    zero_counts_xfmrs_i = list(set(trsf_pole_nbs).difference(set(evsSum_df_i['trsf_pole_nb'].unique())))
    if len(zero_counts_xfmrs_i)>0:
        zero_counts_df_i = OutagePredictor.build_zero_counts_evsSum_df(
            trsf_pole_nbs     = zero_counts_xfmrs_i, 
            evsSum_df         = evsSum_df_i, 
            date              = pred_date_i - self.idk_name_2, 
            evsSum_sql_kwargs = self.evsSum_sql_kwargs, 
            date_col          = 'aep_event_dt',
            trsf_pole_nb_col  = 'trsf_pole_nb', 
            conn_aws          = self.conn_aws, 
            verbose           = True
        )
        #-------------------------
        zero_counts_df = OutagePredictor.append_to_evsSum_df(
            evsSum_df        = zero_counts_df, 
            addtnl_evsSum_df = zero_counts_df_i, 
            sort_by          = None
        )

In [ ]:
zero_counts_df

In [ ]:
evsSum_df

In [ ]:
aww_yeahhh = OutagePredictor.append_to_evsSum_df(
    evsSum_df        = evsSum_df, 
    addtnl_evsSum_df = zero_counts_df, 
    sort_by          = None
)

In [ ]:
aww_yeahhh

In [ ]:
cr_cols = Utilities.find_in_list_with_regex(
    lst=aww_yeahhh.columns.tolist(), 
    regex_pattern=r'cr\d*', 
    ignore_case=False
)

In [ ]:
aww_yeahhh[aww_yeahhh[cr_cols].sum(axis=1)==0]

In [ ]:
np.sum([len(v) for v in zc_xfmrs_by_date.values()])

In [ ]:
zc_xfmrs_by_date

In [ ]:
# predictors = dict()
# for model_i, trsf_poles_i in ledger_inv_srs.items():
#     # if model_i!=2:
#     #     continue
#     print(model_i)
#     model_i_dir = os.path.join(model_base_dir, 'Slices', str(model_i))
#     assert(os.path.isdir(model_i_dir))
#     #-------------------------
#     # If no model able to be made for group, directory will be empty
#     if not os.listdir(model_i_dir):
#         print('No model compiled.\nContinuing to next')
#         continue
#     #-------------------------
#     predictor_i = OutagePredictor(prediction_date=prediction_date)
#     predictor_i.set_model_dir(model_i_dir)
#     predictor_i.set_trsf_pole_nbs(trsf_poles_i)
#     predictor_i.initialize_data(
#         evsSum_sql_fcn=AMIEndEvents_SQL.build_sql_end_events,  
#         evsSum_sql_kwargs=dict(opco='oh'), 
#     )
#     #-------------------------
#     assert(model_i not in predictors.keys())
#     predictors[model_i] = predictor_i
#     #-------------------------
#     zero_counts_xfmrs = list(set(predictor_i.trsf_pole_nbs).difference(set(predictor_i.evsSum_df['trsf_pole_nb'].unique())))
#     zero_counts_df = OutagePredictor.get_xfmr_meter_cnt_closest_to_date(
#         trsf_pole_nbs      = zero_counts_xfmrs, 
#         date               = date, 
#         evsSum_sql_kwargs  = evsSum_sql_kwargs, 
#         include_meter_cols = True, 
#         batch_size         = 1000, 
#         verbose            = True, 
#         n_update           = 10, 
#         conn_aws           = None, 
#         return_sql         = False,
#         keep_extra_cols    = False
#     )
#     #-------------------------
#     zero_counts_df['idx'] = list(range(predictor_i.evsSum_df.shape[0], predictor_i.evsSum_df.shape[0]+zero_counts_df.shape[0]))
#     zero_counts_df = zero_counts_df.set_index('idx', drop=True)
#     zero_counts_df.index.name = None
#     #-------------------------
#     if(
#         predictor_i.evsSum_df.shape[0]>0 and 
#         len(list(set(predictor_i.trsf_pole_nbs).difference(set(predictor_i.evsSum_df['trsf_pole_nb'].unique())))) >0 and
#         zero_counts_df.shape[0]>0
#     ):
#         print(f"{model_i}!!!!!!!!!!!!!!!!")

In [ ]:
assert(0)

In [ ]:
predictors = dict()
for model_i, trsf_poles_i in ledger_inv_srs.items():
    print(model_i)
    model_i_dir = os.path.join(model_base_dir, 'Slices', str(model_i))
    assert(os.path.isdir(model_i_dir))
    #-------------------------
    # If no model able to be made for group, directory will be empty
    if not os.listdir(model_i_dir):
        print('No model compiled.\nContinuing to next')
        continue
    #-------------------------
    predictor_i = OutagePredictor(prediction_date=prediction_date)
    predictor_i.set_model_dir(model_i_dir)
    predictor_i.set_trsf_pole_nbs(trsf_poles_i)
    predictor_i.initialize_data(
        evsSum_sql_fcn=AMIEndEvents_SQL.build_sql_end_events,  
        evsSum_sql_kwargs=dict(opco='oh'), 
    )
    #-------------------------
    assert(model_i not in predictors.keys())
    predictors[model_i] = predictor_i